In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
df_green = spark.read.csv('data/raw/green/*/*', header=True, inferSchema=True)

In [4]:
rdd = df_green \
        .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
        .rdd

In [5]:
from datetime import datetime

In [6]:
start = datetime(year = 2020, month = 1, day = 1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [7]:
rows = rdd.take(10)
row = rows[0]

In [8]:
row

Row(lpep_pickup_datetime=datetime.datetime(2019, 12, 18, 15, 52, 30), PULocationID=264, total_amount=4.81)

In [9]:
def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second = 0, microsecond = 0)
    zone = row.PULocationID
    key = (hour,zone)

    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key,value)

In [10]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value

    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    return (output_amount, output_count)

In [11]:
from collections import namedtuple

In [12]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [13]:
def unwrap (row):
    return RevenueRow(
        hour = row[0][0],
        zone = row[0][1],
        revenue = row[1][0],
        count = row[1][1]
    )

In [14]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF()

In [15]:
df_result.show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-01 00:00:00|  74| 317.0900000000001|   24|
|2020-01-01 00:00:00| 134| 69.05000000000001|    6|
|2020-01-01 00:00:00| 116|328.76000000000005|   18|
|2020-01-01 00:00:00| 260|296.25000000000006|   21|
|2020-01-01 00:00:00| 160|              11.3|    1|
|2020-01-01 01:00:00| 244|255.85000000000002|   20|
|2020-01-01 00:00:00| 146|             99.37|    6|
|2020-01-01 00:00:00|  40|            168.98|    8|
|2020-01-01 00:00:00| 188|            127.57|    6|
|2020-01-01 00:00:00| 196|             37.91|    2|
|2020-01-01 00:00:00| 258|              45.3|    1|
|2020-01-01 00:00:00| 174|              21.7|    1|
|2020-01-01 00:00:00| 236|             13.86|    1|
|2020-01-01 01:00:00|  74|            606.74|   43|
|2020-01-01 02:00:00| 152|220.57000000000002|   11|
|2020-01-01 01:00:00| 256|435.40000000000003|   18|
|2020-01-01 

In [16]:
df_result.schema

StructType([StructField('hour', TimestampType(), True), StructField('zone', LongType(), True), StructField('revenue', DoubleType(), True), StructField('count', LongType(), True)])

In [17]:
df_result.write.parquet('data/report/green-revenue')

AnalysisException: path file:/c:/Users/USER/Code/data-engineering-zoomcamp/05-batch/code/data/report/green-revenue already exists.

In [18]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID','trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [22]:
import pandas as pd

In [23]:
rows = duration_rdd.take(10)

In [28]:
def model_predict(df):
    y_pred = df.trip_distance * 5
    return y_pred

In [29]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns = columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [31]:
duration_rdd.mapPartitions(apply_model_in_batch).toDF().show()

+-----+--------+--------------------+------------+------------+-------------+------------------+
|Index|VendorID|lpep_pickup_datetime|PULocationID|DOLocationID|trip_distance|predicted_duration|
+-----+--------+--------------------+------------+------------+-------------+------------------+
|    0|     2.0|                  {}|         264|         264|          0.0|               0.0|
|    1|     2.0|                  {}|          66|          65|         1.28|               6.4|
|    2|     2.0|                  {}|         181|         228|         2.47|12.350000000000001|
|    3|     1.0|                  {}|         129|         263|          6.3|              31.5|
|    4|     1.0|                  {}|         210|         150|          2.3|              11.5|
|    5|     1.0|                  {}|          35|          39|          3.0|              15.0|
|    6|     2.0|                  {}|          25|          61|         2.77|             13.85|
|    7|     2.0|              